In [ ]:
pip install fpdf

In [ ]:
python --version

In [ ]:
pip install pillow opencv-python pyzbar numpy reportlab


In [ ]:
https://github.com/NaturalHistoryMuseum/pyzbar#windows

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas
import smtplib
from email.message import EmailMessage
import cv2
from pyzbar.pyzbar import decode
import threading
from datetime import datetime
import numpy as np
from PIL import Image, ImageTk

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            barcode TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            weight REAL
        )
    """)
    conn.commit()
    conn.close()

def add_sample_products():
    data = [
        ("12345", "Milk", 50.0, 1.0),
        ("56789", "Bread", 30.0, 0.5),
        ("345678", "Eggs", 60.0, 0.8),
        ("456789", "Rice", 120.0, 2.0)
    ]
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    for item in data:
        cursor.execute("INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?)", item)
    conn.commit()
    conn.close()

# ---------- SMART CART APP ----------
class SmartCartApp:
    def __init__(self, root, customer_data):
        self.root = root
        self.root.title("Smart Cart System")
        self.root.geometry("1100x650")

        self.name = customer_data['name']
        self.phone = customer_data['phone']
        self.email = customer_data['email']
        self.estimate = float(customer_data['estimate'])

        self.cart = []
        self.total_price = 0
        self.total_weight = 0
        self.last_scanned = None

        self.barcode_var = tk.StringVar()
        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Customer: {self.name} | Phone: {self.phone} | Budget: ₹{self.estimate:.2f}", font=("Arial", 12, "bold")).pack(pady=5)

        scan_frame = tk.LabelFrame(self.root, text="Scan Product")
        scan_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(scan_frame, text="Barcode:").pack(side="left", padx=5)
        tk.Entry(scan_frame, textvariable=self.barcode_var).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Add", command=self.add_product).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Scan with Webcam", command=self.start_scanner).pack(side="left", padx=10)

        self.tree = ttk.Treeview(self.root, columns=("Name", "Price", "Weight"), show="headings")
        for col in ("Name", "Price", "Weight"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=5)

        self.summary_label = tk.Label(self.root, text="Total: ₹0.00 | Weight: 0.00 kg", font=("Arial", 12))
        self.summary_label.pack()

        bottom = tk.Frame(self.root)
        bottom.pack(pady=10)
        tk.Button(bottom, text="Generate PDF", command=self.generate_pdf).pack(side="left", padx=10)
        tk.Button(bottom, text="Send Email", command=self.send_email).pack(side="left", padx=10)
        tk.Button(bottom, text="QR Code for Payment", command=self.show_qr_code).pack(side="left", padx=10)

    def add_product(self):
        barcode = self.barcode_var.get().strip()
        conn = sqlite3.connect("smartcart.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name, price, weight FROM products WHERE barcode=?", (barcode,))
        result = cursor.fetchone()
        conn.close()

        if not result:
            messagebox.showerror("Error", "Product not found.")
            return

        name, price, weight = result

        for i, item in enumerate(self.cart):
            if item['barcode'] == barcode:
                response = messagebox.askquestion("Already Added", f"{name} is already in the cart.\nAdd another or remove it?", icon='question')
                if response == 'yes':
                    if self.total_price + price > self.estimate:
                        messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
                        return
                    self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
                    self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
                    self.total_price += price
                    self.total_weight += weight
                else:
                    self.cart.pop(i)
                    for row in self.tree.get_children():
                        if self.tree.item(row)["values"][0] == name:
                            self.tree.delete(row)
                            break
                    self.total_price -= price
                    self.total_weight -= weight
                self.update_summary()
                self.barcode_var.set("")
                return

        if self.total_price + price > self.estimate:
            messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
            return

        self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
        self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
        self.total_price += price
        self.total_weight += weight
        self.update_summary()
        self.barcode_var.set("")

    def update_summary(self):
        self.summary_label.config(text=f"Total: ₹{self.total_price:.2f} | Weight: {self.total_weight:.2f} kg")

    def generate_pdf(self):
        c = canvas.Canvas("invoice.pdf")
        c.drawString(50, 800, f"Customer: {self.name}")
        c.drawString(50, 780, f"Phone: {self.phone}")
        c.drawString(50, 760, f"Email: {self.email}")
        c.drawString(50, 740, f"Estimate: ₹{self.estimate:.2f}")
        c.drawString(50, 720, "Items:")
        y = 700
        for item in self.cart:
            c.drawString(60, y, f"{item['name']} - ₹{item['price']:.2f} - {item['weight']} kg")
            y -= 20
        c.drawString(50, y - 20, f"Total: ₹{self.total_price:.2f}")
        c.drawString(50, y - 40, f"Total Weight: {self.total_weight:.2f} kg")
        c.save()
        messagebox.showinfo("PDF", "Invoice saved as invoice.pdf")

    def send_email(self):
        try:
            msg = EmailMessage()
            msg['Subject'] = "Smart Cart Invoice"
            msg['From'] = "youremail@example.com"
            msg['To'] = self.email
            msg.set_content("Here is your invoice. Thank you!")
            with open("invoice.pdf", 'rb') as f:
                msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename='invoice.pdf')
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login("youremail@example.com", "yourapppassword")
                smtp.send_message(msg)
            messagebox.showinfo("Email", "Invoice sent successfully!")
        except Exception as e:
            messagebox.showerror("Email Error", str(e))

    def show_qr_code(self):
        qr_win = tk.Toplevel(self.root)
        qr_win.title("QR Code for Payment")
        qr_win.geometry("350x400")
        try:
            image = Image.open("smartCart.jpeg")
            image = image.resize((300, 300), Image.Resampling.LANCZOS)
            qr_img = ImageTk.PhotoImage(image)
            label = tk.Label(qr_win, image=qr_img)
            label.image = qr_img
            label.pack(pady=10)
            tk.Label(qr_win, text="Scan to Pay - SmartCart", font=("Arial", 14, "bold")).pack(pady=5)
        except FileNotFoundError:
            tk.Label(qr_win, text="QR Code image 'smartCart.jpeg' not found.", font=("Arial", 12), fg="red").pack(pady=30)

    def start_scanner(self):
        def scan():
            cap = cv2.VideoCapture(1)
            while True:
                _, frame = cap.read()
                barcodes = decode(frame)
                for barcode in barcodes:
                    code = barcode.data.decode('utf-8')
                    now = datetime.now()
                    if not self.last_scanned or (now - self.last_scanned).total_seconds() > 2:
                        self.last_scanned = now
                        self.barcode_var.set(code)
                        self.add_product()
                    pts = barcode.polygon
                    if len(pts) == 4:
                        cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                cv2.imshow("Scan Barcode - Press Q to close", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        threading.Thread(target=scan, daemon=True).start()

# ---------- START WINDOW ----------
def start_screen():
    def proceed():
        name = name_var.get().strip()
        phone = phone_var.get().strip()
        email = email_var.get().strip()
        estimate = estimate_var.get()
        if not (name and phone and email and estimate):
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        start.destroy()
        main = tk.Tk()
        SmartCartApp(main, {
            "name": name,
            "phone": phone,
            "email": email,
            "estimate": estimate
        })
        main.mainloop()

    start = tk.Tk()
    start.title("Enter Customer Details")
    start.geometry("500x350")

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    email_var = tk.StringVar()
    estimate_var = tk.DoubleVar()

    tk.Label(start, text="Enter Your Details", font=("Arial", 16, "bold")).pack(pady=10)

    tk.Label(start, text="Name").pack(pady=5)
    tk.Entry(start, textvariable=name_var).pack()
    tk.Label(start, text="Phone").pack(pady=5)
    tk.Entry(start, textvariable=phone_var).pack()
    tk.Label(start, text="Email").pack(pady=5)
    tk.Entry(start, textvariable=email_var).pack()
    tk.Label(start, text="Estimate (₹)").pack(pady=5)
    tk.Entry(start, textvariable=estimate_var).pack()
    tk.Button(start, text="Proceed", command=proceed).pack(pady=15)

    start.mainloop()

# ---------- MAIN ----------
if __name__ == "__main__":
    init_db()
    add_sample_products()
    start_screen()


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas
import smtplib
from email.message import EmailMessage
import cv2
from pyzbar.pyzbar import decode
import threading
from datetime import datetime
import numpy as np
from PIL import Image, ImageTk
import serial
import time

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            barcode TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            weight REAL
        )
    """)
    conn.commit()
    conn.close()

def add_sample_products():
    data = [
        ("12345", "Milk", 50.0, 1.0),
        ("56789", "Bread", 30.0, 0.5),
        ("345678", "Eggs", 60.0, 0.8),
        ("456789", "Rice", 120.0, 2.0)
    ]
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    for item in data:
        cursor.execute("INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?)", item)
    conn.commit()
    conn.close()

# ---------- SMART CART APP ----------
class SmartCartApp:
    def __init__(self, root, customer_data):
        self.root = root
        self.root.title("Smart Cart System")
        self.root.geometry("1100x650")

        self.name = customer_data['name']
        self.phone = customer_data['phone']
        self.email = customer_data['email']
        self.estimate = float(customer_data['estimate'])

        self.cart = []
        self.total_price = 0
        self.total_weight = 0
        self.last_scanned = None

        self.barcode_var = tk.StringVar()

        try:
            self.arduino = serial.Serial('COM7', 9600, timeout=1)  # Change COM3 to your port
            time.sleep(2)
        except:
            self.arduino = None
            print("Could not connect to Arduino")

        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Customer: {self.name} | Phone: {self.phone} | Budget: ₹{self.estimate:.2f}", font=("Arial", 12, "bold")).pack(pady=5)

        scan_frame = tk.LabelFrame(self.root, text="Scan Product")
        scan_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(scan_frame, text="Barcode:").pack(side="left", padx=5)
        tk.Entry(scan_frame, textvariable=self.barcode_var).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Add", command=self.add_product).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Scan with Webcam", command=self.start_scanner).pack(side="left", padx=10)

        self.tree = ttk.Treeview(self.root, columns=("Name", "Price", "Weight"), show="headings")
        for col in ("Name", "Price", "Weight"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=5)

        self.summary_label = tk.Label(self.root, text="Total: ₹0.00 | Weight: 0.00 kg", font=("Arial", 12))
        self.summary_label.pack()

        bottom = tk.Frame(self.root)
        bottom.pack(pady=10)
        tk.Button(bottom, text="Generate PDF", command=self.generate_pdf).pack(side="left", padx=10)
        tk.Button(bottom, text="Send Email", command=self.send_email).pack(side="left", padx=10)
        tk.Button(bottom, text="QR Code for Payment", command=self.show_qr_code).pack(side="left", padx=10)

    def add_product(self):
        barcode = self.barcode_var.get().strip()
        conn = sqlite3.connect("smartcart.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name, price, weight FROM products WHERE barcode=?", (barcode,))
        result = cursor.fetchone()
        conn.close()

        if not result:
            messagebox.showerror("Error", "Product not found.")
            return

        name, price, weight = result

        for i, item in enumerate(self.cart):
            if item['barcode'] == barcode:
                response = messagebox.askquestion("Already Added", f"{name} is already in the cart.\nAdd another or remove it?", icon='question')
                if response == 'yes':
                    if self.total_price + price > self.estimate:
                        messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
                        return
                    self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
                    self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
                    self.total_price += price
                    self.total_weight += weight
                else:
                    self.cart.pop(i)
                    for row in self.tree.get_children():
                        if self.tree.item(row)["values"][0] == name:
                            self.tree.delete(row)
                            break
                    self.total_price -= price
                    self.total_weight -= weight
                self.update_summary()
                self.barcode_var.set("")
                return

        if self.total_price + price > self.estimate:
            messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
            return

        self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
        self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
        self.total_price += price
        self.total_weight += weight
        self.update_summary()
        self.barcode_var.set("")

    def update_summary(self):
        self.summary_label.config(text=f"Total: ₹{self.total_price:.2f} | Weight: {self.total_weight:.2f} kg")
        if self.arduino:
            try:
                self.arduino.write(f"{self.total_weight:.2f}\n".encode())
            except:
                print("Error writing to Arduino")

    def generate_pdf(self):
        c = canvas.Canvas("invoice.pdf")
        c.drawString(50, 800, f"Customer: {self.name}")
        c.drawString(50, 780, f"Phone: {self.phone}")
        c.drawString(50, 760, f"Email: {self.email}")
        c.drawString(50, 740, f"Estimate: ₹{self.estimate:.2f}")
        c.drawString(50, 720, "Items:")
        y = 700
        for item in self.cart:
            c.drawString(60, y, f"{item['name']} - ₹{item['price']:.2f} - {item['weight']} kg")
            y -= 20
        c.drawString(50, y - 20, f"Total: ₹{self.total_price:.2f}")
        c.drawString(50, y - 40, f"Total Weight: {self.total_weight:.2f} kg")
        c.save()
        messagebox.showinfo("PDF", "Invoice saved as invoice.pdf")

    def send_email(self):
        try:
            msg = EmailMessage()
            msg['Subject'] = "Smart Cart Invoice"
            msg['From'] = "youremail@example.com"
            msg['To'] = self.email
            msg.set_content("Here is your invoice. Thank you!")
            with open("invoice.pdf", 'rb') as f:
                msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename='invoice.pdf')
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login("youremail@example.com", "yourapppassword")
                smtp.send_message(msg)
            messagebox.showinfo("Email", "Invoice sent successfully!")
        except Exception as e:
            messagebox.showerror("Email Error", str(e))

    def show_qr_code(self):
        qr_win = tk.Toplevel(self.root)
        qr_win.title("QR Code for Payment")
        qr_win.geometry("350x400")
        try:
            image = Image.open("smartCart.jpeg")
            image = image.resize((300, 300), Image.Resampling.LANCZOS)
            qr_img = ImageTk.PhotoImage(image)
            label = tk.Label(qr_win, image=qr_img)
            label.image = qr_img
            label.pack(pady=10)
            tk.Label(qr_win, text="Scan to Pay - SmartCart", font=("Arial", 14, "bold")).pack(pady=5)
        except FileNotFoundError:
            tk.Label(qr_win, text="QR Code image 'smartCart.jpeg' not found.", font=("Arial", 12), fg="red").pack(pady=30)

    def start_scanner(self):
        def scan():
            cap = cv2.VideoCapture(1)
            while True:
                _, frame = cap.read()
                barcodes = decode(frame)
                for barcode in barcodes:
                    code = barcode.data.decode('utf-8')
                    now = datetime.now()
                    if not self.last_scanned or (now - self.last_scanned).total_seconds() > 2:
                        self.last_scanned = now
                        self.barcode_var.set(code)
                        self.add_product()
                    pts = barcode.polygon
                    if len(pts) == 4:
                        cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                cv2.imshow("Scan Barcode - Press Q to close", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        threading.Thread(target=scan, daemon=True).start()

# ---------- START WINDOW ----------
def start_screen():
    def proceed():
        name = name_var.get().strip()
        phone = phone_var.get().strip()
        email = email_var.get().strip()
        estimate = estimate_var.get()
        if not (name and phone and email and estimate):
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        start.destroy()
        main = tk.Tk()
        SmartCartApp(main, {
            "name": name,
            "phone": phone,
            "email": email,
            "estimate": estimate
        })
        main.mainloop()

    start = tk.Tk()
    start.title("Enter Customer Details")
    start.geometry("500x350")

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    email_var = tk.StringVar()
    estimate_var = tk.DoubleVar()

    tk.Label(start, text="Enter Your Details", font=("Arial", 16, "bold")).pack(pady=10)

    tk.Label(start, text="Name").pack(pady=5)
    tk.Entry(start, textvariable=name_var).pack()
    tk.Label(start, text="Phone").pack(pady=5)
    tk.Entry(start, textvariable=phone_var).pack()
    tk.Label(start, text="Email").pack(pady=5)
    tk.Entry(start, textvariable=email_var).pack()
    tk.Label(start, text="Estimate (₹)").pack(pady=5)
    tk.Entry(start, textvariable=estimate_var).pack()
    tk.Button(start, text="Proceed", command=proceed).pack(pady=15)

    start.mainloop()

# ---------- MAIN ----------
if __name__ == "__main__":
    init_db()
    add_sample_products()
    start_screen()


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas
import smtplib
from email.message import EmailMessage
import cv2
from pyzbar.pyzbar import decode
import threading
from datetime import datetime
import numpy as np
from PIL import Image, ImageTk
import serial
import time

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            barcode TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            weight REAL
        )
    """)
    conn.commit()
    conn.close()

def add_sample_products():
    data = [
        ("12345", "Milk", 50.0, 1.0),
        ("56789", "Bread", 30.0, 0.5),
        ("345678", "Eggs", 60.0, 0.8),
        ("456789", "Rice", 120.0, 2.0)
    ]
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    for item in data:
        cursor.execute("INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?)", item)
    conn.commit()
    conn.close()

# ---------- SMART CART APP ----------
class SmartCartApp:
    def __init__(self, root, customer_data):
        self.root = root
        self.root.title("Smart Cart System")
        self.root.geometry("1100x650")

        self.name = customer_data['name']
        self.phone = customer_data['phone']
        self.email = customer_data['email']
        self.estimate = float(customer_data['estimate'])

        self.cart = []
        self.total_price = 0
        self.total_weight = 0
        self.last_scanned = None

        self.barcode_var = tk.StringVar()

        # Arduino Integration
        try:
            self.arduino = serial.Serial('COM7', 9600, timeout=1)  # Change COM7 to your port
            time.sleep(2)
        except:
            self.arduino = None
            print("Could not connect to Arduino")

        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Customer: {self.name} | Phone: {self.phone} | Budget: ₹{self.estimate:.2f}", font=("Arial", 12, "bold")).pack(pady=5)

        scan_frame = tk.LabelFrame(self.root, text="Scan Product")
        scan_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(scan_frame, text="Barcode:").pack(side="left", padx=5)
        tk.Entry(scan_frame, textvariable=self.barcode_var).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Add", command=self.add_product).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Scan with Webcam", command=self.start_scanner).pack(side="left", padx=10)

        self.tree = ttk.Treeview(self.root, columns=("Name", "Price", "Weight"), show="headings")
        for col in ("Name", "Price", "Weight"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=5)

        self.summary_label = tk.Label(self.root, text="Total: ₹0.00 | Weight: 0.00 kg", font=("Arial", 12))
        self.summary_label.pack()

        bottom = tk.Frame(self.root)
        bottom.pack(pady=10)
        tk.Button(bottom, text="Generate PDF", command=self.generate_pdf).pack(side="left", padx=10)
        tk.Button(bottom, text="Send Email", command=self.send_email).pack(side="left", padx=10)
        tk.Button(bottom, text="QR Code for Payment", command=self.show_qr_code).pack(side="left", padx=10)

    def add_product(self):
        barcode = self.barcode_var.get().strip()
        conn = sqlite3.connect("smartcart.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name, price, weight FROM products WHERE barcode=?", (barcode,))
        result = cursor.fetchone()
        conn.close()

        if not result:
            messagebox.showerror("Error", "Product not found.")
            return

        name, price, weight = result

        for i, item in enumerate(self.cart):
            if item['barcode'] == barcode:
                response = messagebox.askquestion("Already Added", f"{name} is already in the cart.\nAdd another or remove it?", icon='question')
                if response == 'yes':
                    if self.total_price + price > self.estimate:
                        messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
                        return
                    self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
                    self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
                    self.total_price += price
                    self.total_weight += weight
                else:
                    self.cart.pop(i)
                    for row in self.tree.get_children():
                        if self.tree.item(row)["values"][0] == name:
                            self.tree.delete(row)
                            break
                    self.total_price -= price
                    self.total_weight -= weight
                self.update_summary()
                self.barcode_var.set("")
                return

        if self.total_price + price > self.estimate:
            messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
            return

        self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
        self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
        self.total_price += price
        self.total_weight += weight
        self.update_summary()
        self.barcode_var.set("")

    def update_summary(self):
        self.summary_label.config(text=f"Total: ₹{self.total_price:.2f} | Weight: {self.total_weight:.2f} kg")
        if self.arduino:
            try:
                self.arduino.write(f"{self.total_weight:.2f}\n".encode())
            except:
                print("Error writing to Arduino")

    def generate_pdf(self):
        c = canvas.Canvas("invoice.pdf")
        c.drawString(50, 800, f"Customer: {self.name}")
        c.drawString(50, 780, f"Phone: {self.phone}")
        c.drawString(50, 760, f"Email: {self.email}")
        c.drawString(50, 740, f"Estimate: ₹{self.estimate:.2f}")
        c.drawString(50, 720, "Items:")
        y = 700
        for item in self.cart:
            c.drawString(60, y, f"{item['name']} - ₹{item['price']:.2f} - {item['weight']} kg")
            y -= 20
        c.drawString(50, y - 20, f"Total: ₹{self.total_price:.2f}")
        c.drawString(50, y - 40, f"Total Weight: {self.total_weight:.2f} kg")
        c.save()
        messagebox.showinfo("PDF", "Invoice saved as invoice.pdf")

    def send_email(self):
        try:
            msg = EmailMessage()
            msg['Subject'] = "Smart Cart Invoice"
            msg['From'] = "youremail@example.com"
            msg['To'] = self.email
            msg.set_content("Here is your invoice. Thank you!")
            with open("invoice.pdf", 'rb') as f:
                msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename='invoice.pdf')
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login("youremail@example.com", "yourapppassword")
                smtp.send_message(msg)
            messagebox.showinfo("Email", "Invoice sent successfully!")
        except Exception as e:
            messagebox.showerror("Email Error", str(e))

    def show_qr_code(self):
        qr_win = tk.Toplevel(self.root)
        qr_win.title("QR Code for Payment")
        qr_win.geometry("350x400")
        try:
            image = Image.open("smartCart.jpeg")
            image = image.resize((300, 300), Image.Resampling.LANCZOS)
            qr_img = ImageTk.PhotoImage(image)
            label = tk.Label(qr_win, image=qr_img)
            label.image = qr_img
            label.pack(pady=10)
            tk.Label(qr_win, text="Scan to Pay - SmartCart", font=("Arial", 14, "bold")).pack(pady=5)
        except FileNotFoundError:
            tk.Label(qr_win, text="QR Code image 'smartCart.jpeg' not found.", font=("Arial", 12), fg="red").pack(pady=30)

    def start_scanner(self):
        def scan():
            cap = cv2.VideoCapture(1)
            while True:
                _, frame = cap.read()
                barcodes = decode(frame)
                for barcode in barcodes:
                    code = barcode.data.decode('utf-8')
                    now = datetime.now()
                    if not self.last_scanned or (now - self.last_scanned).total_seconds() > 2:
                        self.last_scanned = now
                        self.barcode_var.set(code)
                        self.add_product()
                    pts = barcode.polygon
                    if len(pts) == 4:
                        cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                cv2.imshow("Scan Barcode - Press Q to close", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        threading.Thread(target=scan, daemon=True).start()

# ---------- START WINDOW ----------
def start_screen():
    def proceed():
        name = name_var.get().strip()
        phone = phone_var.get().strip()
        email = email_var.get().strip()
        estimate = estimate_var.get()
        if not (name and phone and email and estimate):
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        start.destroy()
        main = tk.Tk()
        SmartCartApp(main, {
            "name": name,
            "phone": phone,
            "email": email,
            "estimate": estimate
        })
        main.mainloop()

    start = tk.Tk()
    start.title("Enter Customer Details")
    start.geometry("500x350")

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    email_var = tk.StringVar()
    estimate_var = tk.DoubleVar()

    tk.Label(start, text="Enter Your Details", font=("Arial", 16, "bold")).pack(pady=10)

    tk.Label(start, text="Name").pack(pady=5)
    tk.Entry(start, textvariable=name_var).pack()
    tk.Label(start, text="Phone").pack(pady=5)
    tk.Entry(start, textvariable=phone_var).pack()
    tk.Label(start, text="Email").pack(pady=5)
    tk.Entry(start, textvariable=email_var).pack()
    tk.Label(start, text="Estimate (₹)").pack(pady=5)
    tk.Entry(start, textvariable=estimate_var).pack()
    tk.Button(start, text="Proceed", command=proceed).pack(pady=15)

    start.mainloop()

# ---------- MAIN ----------
if __name__ == "__main__":
    init_db()
    add_sample_products()
    start_screen()


In [ ]:
# Required libraries
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas
import smtplib
from email.message import EmailMessage
import cv2
from pyzbar.pyzbar import decode
import threading
from datetime import datetime
import numpy as np
from PIL import Image, ImageTk
import serial
import time

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            barcode TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            weight REAL
        )
    """)
    conn.commit()
    conn.close()

def add_sample_products():
    data = [
        ("12345", "Milk", 50.0, 1.0),
        ("56789", "Bread", 30.0, 0.5),
        ("345678", "Eggs", 60.0, 0.8),
        ("456789", "Rice", 120.0, 2.0)
    ]
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    for item in data:
        cursor.execute("INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?)", item)
    conn.commit()
    conn.close()

# ---------- SMART CART APP ----------
class SmartCartApp:
    def __init__(self, root, customer_data):
        self.root = root
        self.root.title("Smart Cart System")
        self.root.geometry("1100x650")

        self.name = customer_data['name']
        self.phone = customer_data['phone']
        self.email = customer_data['email']
        self.estimate = float(customer_data['estimate'])

        self.cart = []
        self.total_price = 0
        self.total_weight = 0
        self.last_scanned = None

        self.barcode_var = tk.StringVar()

        try:
            self.arduino = serial.Serial('COM7', 9600, timeout=1)  # Adjust port
            time.sleep(2)
        except:
            self.arduino = None
            print("Could not connect to Arduino")

        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Customer: {self.name} | Phone: {self.phone} | Budget: ₹{self.estimate:.2f}", font=("Arial", 12, "bold")).pack(pady=5)

        scan_frame = tk.LabelFrame(self.root, text="Scan Product")
        scan_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(scan_frame, text="Barcode:").pack(side="left", padx=5)
        tk.Entry(scan_frame, textvariable=self.barcode_var).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Add", command=self.add_product).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Scan with Webcam", command=self.start_scanner).pack(side="left", padx=10)

        self.tree = ttk.Treeview(self.root, columns=("Name", "Price", "Weight"), show="headings")
        for col in ("Name", "Price", "Weight"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=5)

        self.summary_label = tk.Label(self.root, text="Total: ₹0.00 | Weight: 0.00 kg", font=("Arial", 12))
        self.summary_label.pack()

        bottom = tk.Frame(self.root)
        bottom.pack(pady=10)
        tk.Button(bottom, text="Generate PDF", command=self.generate_pdf).pack(side="left", padx=10)
        tk.Button(bottom, text="Send Email", command=self.send_email).pack(side="left", padx=10)
        tk.Button(bottom, text="QR Code for Payment", command=self.show_qr_code).pack(side="left", padx=10)

    def add_product(self):
        barcode = self.barcode_var.get().strip()
        conn = sqlite3.connect("smartcart.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name, price, weight FROM products WHERE barcode=?", (barcode,))
        result = cursor.fetchone()
        conn.close()

        if not result:
            messagebox.showerror("Error", "Product not found.")
            return

        name, price, weight = result
        if self.total_price + price > self.estimate:
            messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
            return

        self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
        self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
        self.total_price += price
        self.total_weight += weight
        self.update_summary(name)
        self.barcode_var.set("")

    def update_summary(self, latest_item=None):
        self.summary_label.config(text=f"Total: ₹{self.total_price:.2f} | Weight: {self.total_weight:.2f} kg")
        if self.arduino:
            try:
                message = f"<ITEM:{latest_item or '-'};PRICE:{self.total_price:.2f};WEIGHT:{self.total_weight:.2f};COUNT:{len(self.cart)}>"
                self.arduino.write(message.encode())
            except Exception as e:
                print(f"Arduino Write Error: {e}")

    def generate_pdf(self):
        c = canvas.Canvas("invoice.pdf")
        c.drawString(50, 800, f"Customer: {self.name}")
        c.drawString(50, 780, f"Phone: {self.phone}")
        c.drawString(50, 760, f"Email: {self.email}")
        c.drawString(50, 740, f"Estimate: ₹{self.estimate:.2f}")
        c.drawString(50, 720, "Items:")
        y = 700
        for item in self.cart:
            c.drawString(60, y, f"{item['name']} - ₹{item['price']:.2f} - {item['weight']} kg")
            y -= 20
        c.drawString(50, y - 20, f"Total: ₹{self.total_price:.2f}")
        c.drawString(50, y - 40, f"Total Weight: {self.total_weight:.2f} kg")
        c.save()
        messagebox.showinfo("PDF", "Invoice saved as invoice.pdf")

    def send_email(self):
        try:
            msg = EmailMessage()
            msg['Subject'] = "Smart Cart Invoice"
            msg['From'] = "youremail@example.com"
            msg['To'] = self.email
            msg.set_content("Here is your invoice. Thank you!")
            with open("invoice.pdf", 'rb') as f:
                msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename='invoice.pdf')
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login("youremail@example.com", "yourapppassword")
                smtp.send_message(msg)
            messagebox.showinfo("Email", "Invoice sent successfully!")
        except Exception as e:
            messagebox.showerror("Email Error", str(e))

    def show_qr_code(self):
        qr_win = tk.Toplevel(self.root)
        qr_win.title("QR Code for Payment")
        qr_win.geometry("350x400")
        try:
            image = Image.open("smartCart.jpeg")
            image = image.resize((300, 300), Image.Resampling.LANCZOS)
            qr_img = ImageTk.PhotoImage(image)
            label = tk.Label(qr_win, image=qr_img)
            label.image = qr_img
            label.pack(pady=10)
            tk.Label(qr_win, text="Scan to Pay - SmartCart", font=("Arial", 14, "bold")).pack(pady=5)
        except FileNotFoundError:
            tk.Label(qr_win, text="QR Code image 'smartCart.jpeg' not found.", font=("Arial", 12), fg="red").pack(pady=30)

    def start_scanner(self):
        def scan():
            cap = cv2.VideoCapture(1)
            while True:
                _, frame = cap.read()
                barcodes = decode(frame)
                for barcode in barcodes:
                    code = barcode.data.decode('utf-8')
                    now = datetime.now()
                    if not self.last_scanned or (now - self.last_scanned).total_seconds() > 2:
                        self.last_scanned = now
                        self.barcode_var.set(code)
                        self.add_product()
                    pts = barcode.polygon
                    if len(pts) == 4:
                        cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                cv2.imshow("Scan Barcode - Press Q to close", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        threading.Thread(target=scan, daemon=True).start()

# ---------- START WINDOW ----------
def start_screen():
    def proceed():
        name = name_var.get().strip()
        phone = phone_var.get().strip()
        email = email_var.get().strip()
        estimate = estimate_var.get()
        if not (name and phone and email and estimate):
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        start.destroy()
        main = tk.Tk()
        SmartCartApp(main, {
            "name": name,
            "phone": phone,
            "email": email,
            "estimate": estimate
        })
        main.mainloop()

    start = tk.Tk()
    start.title("Enter Customer Details")
    start.geometry("500x350")

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    email_var = tk.StringVar()
    estimate_var = tk.DoubleVar()

    tk.Label(start, text="Enter Your Details", font=("Arial", 16, "bold")).pack(pady=10)
    tk.Label(start, text="Name").pack(pady=5)
    tk.Entry(start, textvariable=name_var).pack()
    tk.Label(start, text="Phone").pack(pady=5)
    tk.Entry(start, textvariable=phone_var).pack()
    tk.Label(start, text="Email").pack(pady=5)
    tk.Entry(start, textvariable=email_var).pack()
    tk.Label(start, text="Estimate (₹)").pack(pady=5)
    tk.Entry(start, textvariable=estimate_var).pack()
    tk.Button(start, text="Proceed", command=proceed).pack(pady=15)

    start.mainloop()

# ---------- MAIN ----------
if __name__ == "__main__":
    init_db()
    add_sample_products()
    start_screen()


In [ ]:
# Required libraries
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from reportlab.pdfgen import canvas
import smtplib
from email.message import EmailMessage
import cv2
from pyzbar.pyzbar import decode
import threading
from datetime import datetime
import numpy as np
from PIL import Image, ImageTk
import serial
import time

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            barcode TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            weight REAL
        )
    """)
    conn.commit()
    conn.close()

def add_sample_products():
    data = [
        ("12345", "Milk", 50.0, 1.0),
        ("56789", "Bread", 30.0, 0.5),
        ("345678", "Eggs", 60.0, 0.8),
        ("456789", "Rice", 120.0, 2.0)
    ]
    conn = sqlite3.connect("smartcart.db")
    cursor = conn.cursor()
    for item in data:
        cursor.execute("INSERT OR IGNORE INTO products VALUES (?, ?, ?, ?)", item)
    conn.commit()
    conn.close()

# ---------- SMART CART APP ----------
class SmartCartApp:
    def __init__(self, root, customer_data):
        self.root = root
        self.root.title("Smart Cart System")
        self.root.geometry("1100x650")

        self.name = customer_data['name']
        self.phone = customer_data['phone']
        self.email = customer_data['email']
        self.estimate = float(customer_data['estimate'])

        self.cart = []
        self.total_price = 0
        self.total_weight = 0
        self.last_scanned = None

        self.barcode_var = tk.StringVar()

        try:
            self.arduino = serial.Serial('COM8', 9600, timeout=1)  # Adjust port
            time.sleep(2)
        except:
            self.arduino = None
            print("Could not connect to Arduino")

        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text=f"Customer: {self.name} | Phone: {self.phone} | Budget: ₹{self.estimate:.2f}", font=("Arial", 12, "bold")).pack(pady=5)

        scan_frame = tk.LabelFrame(self.root, text="Scan Product")
        scan_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(scan_frame, text="Barcode:").pack(side="left", padx=5)
        tk.Entry(scan_frame, textvariable=self.barcode_var).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Add", command=self.add_product).pack(side="left", padx=5)
        tk.Button(scan_frame, text="Scan with Webcam", command=self.start_scanner).pack(side="left", padx=10)

        self.tree = ttk.Treeview(self.root, columns=("Name", "Price", "Weight"), show="headings")
        for col in ("Name", "Price", "Weight"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=5)

        self.summary_label = tk.Label(self.root, text="Total: ₹0.00 | Weight: 0.00 kg", font=("Arial", 12))
        self.summary_label.pack()

        bottom = tk.Frame(self.root)
        bottom.pack(pady=10)
        tk.Button(bottom, text="Generate PDF", command=self.generate_pdf).pack(side="left", padx=10)
        tk.Button(bottom, text="Send Email", command=self.send_email).pack(side="left", padx=10)
        tk.Button(bottom, text="QR Code for Payment", command=self.show_qr_code).pack(side="left", padx=10)

    def add_product(self):
        barcode = self.barcode_var.get().strip()
        conn = sqlite3.connect("smartcart.db")
        cursor = conn.cursor()
        cursor.execute("SELECT name, price, weight FROM products WHERE barcode=?", (barcode,))
        result = cursor.fetchone()
        conn.close()

        if not result:
            messagebox.showerror("Error", "Product not found.")
            return

        name, price, weight = result

        # Check if the item already exists in the cart
        existing_item = None
        for item in self.cart:
            if item['barcode'] == barcode:
                existing_item = item
                break

        if existing_item:
            # Prompt user if they want to keep or delete the item
            response = messagebox.askquestion(
                "Item already added",
                f"{name} is already in the cart. Do you want to keep it or remove it?",
                icon="warning"
            )
            if response == 'no':
                # Remove the item from the cart
                self.cart.remove(existing_item)
                self.tree.delete(self.tree.get_children())  # Clear treeview
                for item in self.cart:
                    self.tree.insert('', 'end', values=(item['name'], f"₹{item['price']:.2f}", f"{item['weight']:.2f} kg"))
                self.total_price -= existing_item['price']
                self.total_weight -= existing_item['weight']
                self.update_summary()
                return

        if self.total_price + price > self.estimate:
            messagebox.showwarning("Budget Exceeded", f"{name} not added. Budget would be exceeded.")
            return

        self.cart.append({"barcode": barcode, "name": name, "price": price, "weight": weight})
        self.tree.insert('', 'end', values=(name, f"₹{price:.2f}", f"{weight:.2f} kg"))
        self.total_price += price
        self.total_weight += weight
        self.update_summary(name)
        self.barcode_var.set("")

    def update_summary(self, latest_item=None):
        self.summary_label.config(text=f"Total: ₹{self.total_price:.2f} | Weight: {self.total_weight:.2f} kg")
        if self.arduino:
            try:
                message = f"<ITEM:{latest_item or '-'};PRICE:{self.total_price:.2f};WEIGHT:{self.total_weight:.2f};COUNT:{len(self.cart)}>"
                self.arduino.write(message.encode())
            except Exception as e:
                print(f"Arduino Write Error: {e}")

    def generate_pdf(self):
        c = canvas.Canvas("invoice.pdf")
        c.drawString(50, 800, f"Customer: {self.name}")
        c.drawString(50, 780, f"Phone: {self.phone}")
        c.drawString(50, 760, f"Email: {self.email}")
        c.drawString(50, 740, f"Estimate: ₹{self.estimate:.2f}")
        c.drawString(50, 720, "Items:")
        y = 700
        for item in self.cart:
            c.drawString(60, y, f"{item['name']} - ₹{item['price']:.2f} - {item['weight']} kg")
            y -= 20
        c.drawString(50, y - 20, f"Total: ₹{self.total_price:.2f}")
        c.drawString(50, y - 40, f"Total Weight: {self.total_weight:.2f} kg")
        c.save()
        messagebox.showinfo("PDF", "Invoice saved as invoice.pdf")

    def send_email(self):
        try:
            msg = EmailMessage()
            msg['Subject'] = "Smart Cart Invoice"
            msg['From'] = "youremail@example.com"
            msg['To'] = self.email
            msg.set_content("Here is your invoice. Thank you!")
            with open("invoice.pdf", 'rb') as f:
                msg.add_attachment(f.read(), maintype='application', subtype='pdf', filename='invoice.pdf')
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login("youremail@example.com", "yourapppassword")
                smtp.send_message(msg)
            messagebox.showinfo("Email", "Invoice sent successfully!")
        except Exception as e:
            messagebox.showerror("Email Error", str(e))

    def show_qr_code(self):
        qr_win = tk.Toplevel(self.root)
        qr_win.title("QR Code for Payment")
        qr_win.geometry("350x400")
        try:
            image = Image.open("smartCart.jpeg")
            image = image.resize((300, 300), Image.Resampling.LANCZOS)
            qr_img = ImageTk.PhotoImage(image)
            label = tk.Label(qr_win, image=qr_img)
            label.image = qr_img
            label.pack(pady=10)
            tk.Label(qr_win, text="Scan to Pay - SmartCart", font=("Arial", 14, "bold")).pack(pady=5)
        except FileNotFoundError:
            tk.Label(qr_win, text="QR Code image 'smartCart.jpeg' not found.", font=("Arial", 12), fg="red").pack(pady=30)

    def start_scanner(self):
        def scan():
            cap = cv2.VideoCapture(1)
            while True:
                _, frame = cap.read()
                barcodes = decode(frame)
                for barcode in barcodes:
                    code = barcode.data.decode('utf-8')
                    now = datetime.now()
                    if not self.last_scanned or (now - self.last_scanned).total_seconds() > 2:
                        self.last_scanned = now
                        self.barcode_var.set(code)
                        self.add_product()
                    pts = barcode.polygon
                    if len(pts) == 4:
                        cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                cv2.imshow("Scan Barcode - Press Q to close", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            cap.release()
            cv2.destroyAllWindows()
        threading.Thread(target=scan, daemon=True).start()

# ---------- START WINDOW ----------
def start_screen():
    def proceed():
        name = name_var.get().strip()
        phone = phone_var.get().strip()
        email = email_var.get().strip()
        estimate = estimate_var.get()
        if not (name and phone and email and estimate):
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return
        start.destroy()
        main = tk.Tk()
        SmartCartApp(main, {
            "name": name,
            "phone": phone,
            "email": email,
            "estimate": estimate
        })
        main.mainloop()

    start = tk.Tk()
    start.title("Enter Customer Details")
    start.geometry("500x350")

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    email_var = tk.StringVar()
    estimate_var = tk.DoubleVar()

    tk.Label(start, text="Enter Your Details", font=("Arial", 16, "bold")).pack(pady=10)
    tk.Label(start, text="Name").pack(pady=5)
    tk.Entry(start, textvariable=name_var).pack()
    tk.Label(start, text="Phone").pack(pady=5)
    tk.Entry(start, textvariable=phone_var).pack()
    tk.Label(start, text="Email").pack(pady=5)
    tk.Entry(start, textvariable=email_var).pack()
    tk.Label(start, text="Estimate (₹)").pack(pady=5)
    tk.Entry(start, textvariable=estimate_var).pack()
    tk.Button(start, text="Proceed", command=proceed).pack(pady=15)

    start.mainloop()

# ---------- MAIN ----------
if __name__ == "__main__":
    init_db()
    add_sample_products()
    start_screen()
